# DocTAPE

DocTAPE (Documentation Testing and Automated Placement of Expressions) is a collection of utility functions (and wrappers for [Glue](https://myst-nb.readthedocs.io/en/latest/render/glue.html)) that are useful
for automating the process of building and testing documentation to ensure that documentation doesn't get stale.

Our standard practice it to include a comment (`# Testing Cell`) at the begining of code cells as well as make use of the `remove-cell` tag.

>   "metadata": { "tags": [ "remove-cell" ] },

<details><summary>More info about adding cell tags</summary>

- [Jupyter Book](https://jupyterbook.org/en/stable/content/metadata.html)
- [Visual Studio Code](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.vscode-jupyter-cell-tags)
</details>


In [1]:
# Testing Cell


In [2]:
# Testing Cell

from aviary.utils import doctape
import inspect

imported_functions = {k:v for k,v in inspect.getmembers(doctape, inspect.isfunction) if v.__module__ == doctape.__name__}
imported_classes = {k:v for k,v in inspect.getmembers(doctape, inspect.isclass) if v.__module__ == doctape.__name__}

custom_classes = {
    "expected_error": "is an execption that can be used in try/except blocks to allow desired errors to pass while still raising unexpected errors.",
}
testing_functions = {
    "check_value": "is a simple function for comparing two values",
    "check_contains": "confirms that all the elements of one iterable are contained in the other",
    "check_args": "gets the signature of a function and compares it to the arguments you are expecting",
    "run_command_no_file_error": "executes a CLI command but won't fail if a FileNotFoundError is raised",
}
glue_functions = {
    "glue_variable": "glue a variable for later use in markdown cells of notebooks (can auto format for code)",
    "glue_keys": "recursively glue all of the keys from a dict of dicts",
    "glue_actions": "glue all Aviary CLI options for a given command",
    "glue_class_functions": "glue all class functions for a gen class",
    "glue_function_arguments": "glue all function arguments for a given function",
}
utility_functions = {
    "gramatical_list": "combines the elements of a list into a string with proper punctuation",
    "get_variable_name": "returns the name of the variable passed to the function as a string",
    "get_previous_line": "returns the previous line of code as a string",
    "get_attribute_name": "gets the name of an object's attribute based on it's value",
    "get_all_keys": "recursively get all of the keys from a dict of dicts",
    "get_value": "recursively get a value from a dict of dicts",
    "get_class_names": "gets all class names from a given file and return as a set",
    "get_function_names": "gets all function names in a given file and return as a set",
}

doctape.check_value(imported_classes.keys(),custom_classes.keys())
doctape.check_value(imported_functions.keys(), {
    **testing_functions, **glue_functions, **utility_functions}.keys())

class_list = ''
for key,val in custom_classes.items():
    doctape.glue_variable(key, md_code=True)
    class_list += f'- `{key}` {val}\n'

utility_list = '```{eval-rst}\n'
for key in utility_functions:
    doctape.glue_variable(key, md_code=True)
    utility_list += ' '*4+f'.. autofunction:: aviary.utils.doctape.{key}\n{" "*8}:noindex:\n\n'
utility_list += '```'

testing_list = '```{eval-rst}\n'
for key in testing_functions:
    doctape.glue_variable(key, md_code=True)
    testing_list += ' '*4+f'.. autofunction:: aviary.utils.doctape.{key}\n{" "*8}:noindex:\n\n'
testing_list += '```'

glue_list = '```{eval-rst}\n'
for key in glue_functions:
    doctape.glue_variable(key, md_code=True)
    glue_list += ' '*4+f'.. autofunction:: aviary.utils.doctape.{key}\n{" "*8}:noindex:\n\n'
glue_list += '```'

doctape.glue_variable('class_list', class_list)
doctape.glue_variable('utility_list', utility_list)
doctape.glue_variable('testing_list', testing_list)
doctape.glue_variable('glue_list', glue_list)

ValueError: dict_keys(['check_args', 'check_contains', 'check_value', 'get_all_keys', 'get_attribute_name', 'get_class_names', 'get_function_names', 'get_previous_line', 'get_value', 'get_variable_name', 'glue_actions', 'glue_class_functions', 'glue_function_arguments', 'glue_keys', 'glue_variable', 'gramatical_list', 'run_command_no_file_error']) is not equal to dict_keys(['check_value', 'check_contains', 'check_args', 'run_command_no_file_error', 'glue_variable', 'glue_keys', 'gramatical_list', 'get_variable_name', 'get_previous_line', 'get_attribute_name', 'get_all_keys', 'get_value', 'get_class_names', 'get_function_names'])

## Classes
```{glue:md} class_list
:format: myst
```

## Testing Functions

Functions that raise an error provide the option to specify an error type to use instead of the default. This allows users to change the error type that is raised which can be useful in try/except blocks, especially when combined with the {glue:md}`expected_error` class.

```{glue:md} testing_list
:format: myst
```

In [3]:
# Testing Cell
import myst_nb
from aviary.utils.doctape import glue_variable

glue_variable(myst_nb.__name__)
glue_variable(myst_nb.glue.__name__, md_code=True)

myst_nb

`glue`

## Glue Functions

The glue functions provide a wrapper for the {glue:md}`myst_nb` {glue:md}`glue` function that provides a simplified interface.

```{glue:md} glue_list
:format: myst
```

## Utility Functions

Utility functions are provided that the user may find useful for generating or testing their documentation.

```{glue:md} utility_list
:format: myst
```